# Part 1

## やること

1. Assimilate the business scenario and articulate testable hypotheses.（ビジネスシナリオを同化し、テスト可能な仮説を明確にします。）
2. State the ideal data to address the business opportunity and clarify the rationale for needing specific data.（ビジネスチャンスに対処するための理想的なデータを述べ、特定のデータが必要な理由を明確にします。）
3. Create a python script to extract relevant data from multiple data sources, automating the process of data ingestion.（複数のデータソースから関連データを抽出するPythonスクリプトを作成し、データの取り込みプロセスを自動化します。）
4. Investigate the relationship between the relevant data, the target and the business metric.（関連するデータ、ターゲット、およびビジネス指標の間の関係を調査します。）
5. Articulate your findings using a deliverable with visualizations.（視覚化された成果物を使用して、調査結果を明確にします。）

## import packages

In [1]:
import sys
sys.path.append("../solution-guidance/")

In [2]:
import cslib

## parameter settings

In [3]:
path_data = "../data/"
path_data_production = path_data + "cs-production/"
path_data_train = path_data + "cs-train/"

## load data

In [4]:
# データの取得
cslib.fetch_data(path_data_train)

,country,customer_id,day,invoice,month,price,stream_id,times_viewed,year,invoice_date
0,United Kingdom,13085.0,28,489434,11,6.95,85048,12,2017,2017-11-28
1,United Kingdom,13085.0,28,489434,11,6.75,79323W,12,2017,2017-11-28
2,United Kingdom,13085.0,28,489434,11,2.10,22041,21,2017,2017-11-28
3,United Kingdom,13085.0,28,489434,11,1.25,21232,5,2017,2017-11-28
4,United Kingdom,13085.0,28,489434,11,1.65,22064,17,2017,2017-11-28
...,...,...,...,...,...,...,...,...,...,...
815006,United Kingdom,16098.0,31,562271,7,3.75,22725,2,2019,2019-07-31
815007,United Kingdom,16098.0,31,562271,7,3.75,22726,12,2019,2019-07-31
815008,United Kingdom,16098.0,31,562271,7,3.75,22727,6,2019,2019-07-31
815009,United Kingdom,14056.0,31,562269,7,2.95,22090,2,2019,2019-07-31


In [6]:
cslib.fetch_ts(path_data_train)

... loading ts data from files


{'all':            date  purchases  unique_invoices  unique_streams  total_views  \
 0    2017-11-01          0                0               0            0   
 1    2017-11-02          0                0               0            0   
 2    2017-11-03          0                0               0            0   
 3    2017-11-04          0                0               0            0   
 4    2017-11-05          0                0               0            0   
 ..          ...        ...              ...             ...          ...   
 602  2019-06-26       1358               67             999         6420   
 603  2019-06-27       1620               80             944         9435   
 604  2019-06-28       1027               70             607         5539   
 605  2019-06-29          0                0               0            0   
 606  2019-06-30        602               27             423         2534   
 
     year_month  revenue  
 0      2017-11     0.00  
 1      2017-